# Python HANA ML APL

<div class="alert alert-block alert-info">
<b>Get JSON code of predictive model for standalone apply.</b> <br>
</div>

## Learn from historical data

### Create an HANA Dataframe for the training data

In [ ]:
from hana_ml import dataframe as hd
conn = hd.ConnectionContext(userkey='MLMDA_KEY')
sql_cmd = 'SELECT * FROM "APL_SAMPLES"."AUTO_CLAIMS_FRAUD" ORDER BY CLAIM_ID'
hdf_train = hd.DataFrame(conn, sql_cmd)

In [ ]:
hdf_train.head(6).collect()

### Fit with APL Gradient Boosting

In [ ]:
from hana_ml.algorithms.apl.gradient_boosting_classification import GradientBoostingBinaryClassifier
apl_model = GradientBoostingBinaryClassifier()
apl_model.set_params(other_train_apl_aliases={'APL/VariableAutoSelection':'true'})
apl_model.fit(hdf_train, label='IS_FRAUD', key='CLAIM_ID')

In [ ]:
my_filter = "\"Partition\" = 'Validation' and \"Indicator\" in ('AUC','F1 Score','Cohen''s kappa')"
df = apl_model.get_debrief_report('ClassificationRegression_Performance').filter(my_filter).collect()
df.drop('Oid', axis=1, inplace=True)
format_dict = {'Value':'{:,.3f}'}
df.style.format(format_dict).hide(axis='index')

### Export Apply Code

In [ ]:
apl_scoring_equation = apl_model.export_apply_code(code_type='JSON')

In [ ]:
text_file = open("apl_model.json", "w")
text_file.write(apl_scoring_equation)
text_file.close()